 # Notebook for Downloading ds084.1 Data.

#### Start by downloading some sample data

In [ ]:
import sys, os
import requests

 #### Since RDA requires authentication to access data, the following code blocks will start this process.

In [ ]:
# Compatibility for python2 and getpass not installed
try:
    import getpass
    input = getpass.getpass
except:
    try:
        input = raw_input
    except:
        pass

#### Now, we need your RDA email (username) and password.

In [ ]:
pswd = input('password: ')
email = 'EXAMPLE@test.com'

In [ ]:
values = {'email' : email, 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

In [ ]:
ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

In [ ]:
dspath = 'https://rda.ucar.edu/data/ds084.1/'
filelist = ['2020/20200106/gfs.0p25.2020010600.f024.grib2', '2020/20200106/gfs.0p25.2020010600.f027.grib2', '2020/20200106/gfs.0p25.2020010600.f030.grib2', '2020/20200106/gfs.0p25.2020010600.f033.grib2', '2020/20200106/gfs.0p25.2020010600.f036.grib2', '2020/20200106/gfs.0p25.2020010600.f039.grib2', '2020/20200106/gfs.0p25.2020010600.f042.grib2', '2020/20200106/gfs.0p25.2020010600.f045.grib2', '2020/20200106/gfs.0p25.2020010600.f048.grib2', '2020/20200106/gfs.0p25.2020010600.f051.grib2', '2020/20200106/gfs.0p25.2020010600.f054.grib2', '2020/20200106/gfs.0p25.2020010600.f057.grib2', '2020/20200106/gfs.0p25.2020010600.f060.grib2', '2020/20200106/gfs.0p25.2020010600.f063.grib2', '2020/20200106/gfs.0p25.2020010600.f066.grib2', '2020/20200106/gfs.0p25.2020010600.f069.grib2', '2020/20200106/gfs.0p25.2020010600.f072.grib2', '2020/20200106/gfs.0p25.2020010600.f075.grib2', '2020/20200106/gfs.0p25.2020010600.f078.grib2', '2020/20200106/gfs.0p25.2020010600.f081.grib2', '2020/20200106/gfs.0p25.2020010600.f084.grib2', '2020/20200106/gfs.0p25.2020010600.f087.grib2', '2020/20200106/gfs.0p25.2020010600.f090.grib2', '2020/20200106/gfs.0p25.2020010600.f093.grib2', '2020/20200106/gfs.0p25.2020010600.f096.grib2', '2020/20200106/gfs.0p25.2020010600.f099.grib2', '2020/20200106/gfs.0p25.2020010600.f102.grib2', '2020/20200106/gfs.0p25.2020010600.f105.grib2', '2020/20200106/gfs.0p25.2020010600.f108.grib2', '2020/20200106/gfs.0p25.2020010600.f111.grib2', '2020/20200106/gfs.0p25.2020010600.f114.grib2', '2020/20200106/gfs.0p25.2020010600.f117.grib2', '2020/20200106/gfs.0p25.2020010600.f120.grib2', '2020/20200106/gfs.0p25.2020010600.f123.grib2', '2020/20200106/gfs.0p25.2020010600.f126.grib2', '2020/20200106/gfs.0p25.2020010600.f129.grib2', '2020/20200106/gfs.0p25.2020010600.f132.grib2', '2020/20200106/gfs.0p25.2020010600.f135.grib2', '2020/20200106/gfs.0p25.2020010600.f138.grib2', '2020/20200106/gfs.0p25.2020010600.f141.grib2', '2020/20200106/gfs.0p25.2020010600.f144.grib2', '2020/20200106/gfs.0p25.2020010600.f147.grib2', '2020/20200106/gfs.0p25.2020010600.f150.grib2', '2020/20200106/gfs.0p25.2020010600.f153.grib2', '2020/20200106/gfs.0p25.2020010600.f156.grib2', '2020/20200106/gfs.0p25.2020010600.f159.grib2', '2020/20200106/gfs.0p25.2020010600.f162.grib2', '2020/20200106/gfs.0p25.2020010600.f165.grib2', '2020/20200106/gfs.0p25.2020010600.f168.grib2', '2020/20200106/gfs.0p25.2020010600.f171.grib2', '2020/20200106/gfs.0p25.2020010600.f174.grib2', '2020/20200106/gfs.0p25.2020010600.f177.grib2', '2020/20200106/gfs.0p25.2020010600.f180.grib2', '2020/20200106/gfs.0p25.2020010600.f183.grib2', '2020/20200106/gfs.0p25.2020010600.f186.grib2', '2020/20200106/gfs.0p25.2020010600.f189.grib2', '2020/20200106/gfs.0p25.2020010600.f192.grib2', '2020/20200106/gfs.0p25.2020010600.f195.grib2', '2020/20200106/gfs.0p25.2020010600.f198.grib2', '2020/20200106/gfs.0p25.2020010600.f201.grib2', '2020/20200106/gfs.0p25.2020010600.f204.grib2', '2020/20200106/gfs.0p25.2020010600.f207.grib2', '2020/20200106/gfs.0p25.2020010600.f210.grib2', '2020/20200106/gfs.0p25.2020010600.f213.grib2', '2020/20200106/gfs.0p25.2020010600.f216.grib2', '2020/20200106/gfs.0p25.2020010600.f219.grib2', '2020/20200106/gfs.0p25.2020010600.f222.grib2', '2020/20200106/gfs.0p25.2020010600.f225.grib2', '2020/20200106/gfs.0p25.2020010600.f228.grib2', '2020/20200106/gfs.0p25.2020010600.f231.grib2', '2020/20200106/gfs.0p25.2020010600.f234.grib2', '2020/20200106/gfs.0p25.2020010600.f237.grib2', '2020/20200106/gfs.0p25.2020010600.f240.grib2', '2020/20200106/gfs.0p25.2020010600.f252.grib2', '2020/20200106/gfs.0p25.2020010600.f264.grib2', '2020/20200106/gfs.0p25.2020010600.f276.grib2', '2020/20200106/gfs.0p25.2020010600.f288.grib2', '2020/20200106/gfs.0p25.2020010600.f300.grib2', '2020/20200106/gfs.0p25.2020010600.f312.grib2', '2020/20200106/gfs.0p25.2020010600.f324.grib2', '2020/20200106/gfs.0p25.2020010600.f336.grib2', '2020/20200106/gfs.0p25.2020010600.f348.grib2', '2020/20200106/gfs.0p25.2020010600.f360.grib2', '2020/20200106/gfs.0p25.2020010600.f372.grib2', '2020/20200106/gfs.0p25.2020010600.f384.grib2', '2020/20200106/gfs.0p25.2020010606.f000.grib2', '2020/20200106/gfs.0p25.2020010606.f003.grib2', '2020/20200106/gfs.0p25.2020010606.f006.grib2', '2020/20200106/gfs.0p25.2020010606.f009.grib2', '2020/20200106/gfs.0p25.2020010606.f012.grib2', '2020/20200106/gfs.0p25.2020010606.f015.grib2', '2020/20200106/gfs.0p25.2020010606.f018.grib2', '2020/20200106/gfs.0p25.2020010606.f021.grib2', '2020/20200106/gfs.0p25.2020010606.f024.grib2', '2020/20200106/gfs.0p25.2020010606.f027.grib2', '2020/20200106/gfs.0p25.2020010606.f030.grib2', '2020/20200106/gfs.0p25.2020010606.f033.grib2', '2020/20200106/gfs.0p25.2020010606.f036.grib2', '2020/20200106/gfs.0p25.2020010606.f039.grib2', '2020/20200106/gfs.0p25.2020010606.f042.grib2', '2020/20200106/gfs.0p25.2020010606.f045.grib2', '2020/20200106/gfs.0p25.2020010606.f048.grib2', '2020/20200106/gfs.0p25.2020010606.f051.grib2', '2020/20200106/gfs.0p25.2020010606.f054.grib2', '2020/20200106/gfs.0p25.2020010606.f057.grib2', '2020/20200106/gfs.0p25.2020010606.f060.grib2', '2020/20200106/gfs.0p25.2020010606.f063.grib2', '2020/20200106/gfs.0p25.2020010606.f066.grib2', '2020/20200106/gfs.0p25.2020010606.f069.grib2', '2020/20200106/gfs.0p25.2020010606.f072.grib2', '2020/20200106/gfs.0p25.2020010606.f075.grib2', '2020/20200106/gfs.0p25.2020010606.f078.grib2', '2020/20200106/gfs.0p25.2020010606.f081.grib2', '2020/20200106/gfs.0p25.2020010606.f084.grib2', '2020/20200106/gfs.0p25.2020010606.f087.grib2', '2020/20200106/gfs.0p25.2020010606.f090.grib2', '2020/20200106/gfs.0p25.2020010606.f093.grib2', '2020/20200106/gfs.0p25.2020010606.f096.grib2', '2020/20200106/gfs.0p25.2020010606.f099.grib2', '2020/20200106/gfs.0p25.2020010606.f102.grib2', '2020/20200106/gfs.0p25.2020010606.f105.grib2', '2020/20200106/gfs.0p25.2020010606.f108.grib2', '2020/20200106/gfs.0p25.2020010606.f111.grib2', '2020/20200106/gfs.0p25.2020010606.f114.grib2', '2020/20200106/gfs.0p25.2020010606.f117.grib2', '2020/20200106/gfs.0p25.2020010606.f120.grib2', '2020/20200106/gfs.0p25.2020010606.f123.grib2', '2020/20200106/gfs.0p25.2020010606.f126.grib2', '2020/20200106/gfs.0p25.2020010606.f129.grib2', '2020/20200106/gfs.0p25.2020010606.f132.grib2', '2020/20200106/gfs.0p25.2020010606.f135.grib2', '2020/20200106/gfs.0p25.2020010606.f138.grib2', '2020/20200106/gfs.0p25.2020010606.f141.grib2', '2020/20200106/gfs.0p25.2020010606.f144.grib2', '2020/20200106/gfs.0p25.2020010606.f147.grib2', '2020/20200106/gfs.0p25.2020010606.f150.grib2', '2020/20200106/gfs.0p25.2020010606.f153.grib2', '2020/20200106/gfs.0p25.2020010606.f156.grib2', '2020/20200106/gfs.0p25.2020010606.f159.grib2', '2020/20200106/gfs.0p25.2020010606.f162.grib2', '2020/20200106/gfs.0p25.2020010606.f165.grib2', '2020/20200106/gfs.0p25.2020010606.f168.grib2', '2020/20200106/gfs.0p25.2020010606.f171.grib2', '2020/20200106/gfs.0p25.2020010606.f174.grib2', '2020/20200106/gfs.0p25.2020010606.f177.grib2', '2020/20200106/gfs.0p25.2020010606.f180.grib2', '2020/20200106/gfs.0p25.2020010606.f183.grib2', '2020/20200106/gfs.0p25.2020010606.f186.grib2', '2020/20200106/gfs.0p25.2020010606.f189.grib2', '2020/20200106/gfs.0p25.2020010606.f192.grib2', '2020/20200106/gfs.0p25.2020010606.f195.grib2', '2020/20200106/gfs.0p25.2020010606.f198.grib2', '2020/20200106/gfs.0p25.2020010606.f201.grib2', '2020/20200106/gfs.0p25.2020010606.f204.grib2', '2020/20200106/gfs.0p25.2020010606.f207.grib2', '2020/20200106/gfs.0p25.2020010606.f210.grib2', '2020/20200106/gfs.0p25.2020010606.f213.grib2', '2020/20200106/gfs.0p25.2020010606.f216.grib2', '2020/20200106/gfs.0p25.2020010606.f219.grib2', '2020/20200106/gfs.0p25.2020010606.f222.grib2', '2020/20200106/gfs.0p25.2020010606.f225.grib2', '2020/20200106/gfs.0p25.2020010606.f228.grib2', '2020/20200106/gfs.0p25.2020010606.f231.grib2', '2020/20200106/gfs.0p25.2020010606.f234.grib2', '2020/20200106/gfs.0p25.2020010606.f237.grib2', '2020/20200106/gfs.0p25.2020010606.f240.grib2', '2020/20200106/gfs.0p25.2020010606.f252.grib2', '2020/20200106/gfs.0p25.2020010606.f264.grib2', '2020/20200106/gfs.0p25.2020010606.f276.grib2', '2020/20200106/gfs.0p25.2020010606.f288.grib2', '2020/20200106/gfs.0p25.2020010606.f300.grib2', '2020/20200106/gfs.0p25.2020010606.f312.grib2', '2020/20200106/gfs.0p25.2020010606.f324.grib2', '2020/20200106/gfs.0p25.2020010606.f336.grib2', '2020/20200106/gfs.0p25.2020010606.f348.grib2', '2020/20200106/gfs.0p25.2020010606.f360.grib2', '2020/20200106/gfs.0p25.2020010606.f372.grib2', '2020/20200106/gfs.0p25.2020010606.f384.grib2', '2020/20200106/gfs.0p25.2020010612.f000.grib2', '2020/20200106/gfs.0p25.2020010612.f003.grib2', '2020/20200106/gfs.0p25.2020010612.f006.grib2', '2020/20200106/gfs.0p25.2020010612.f009.grib2', '2020/20200106/gfs.0p25.2020010612.f012.grib2', '2020/20200106/gfs.0p25.2020010612.f015.grib2', '2020/20200106/gfs.0p25.2020010612.f018.grib2', '2020/20200106/gfs.0p25.2020010612.f021.grib2', '2020/20200106/gfs.0p25.2020010612.f024.grib2', '2020/20200106/gfs.0p25.2020010612.f027.grib2', '2020/20200106/gfs.0p25.2020010612.f030.grib2', '2020/20200106/gfs.0p25.2020010612.f033.grib2', '2020/20200106/gfs.0p25.2020010612.f036.grib2', '2020/20200106/gfs.0p25.2020010612.f039.grib2', '2020/20200106/gfs.0p25.2020010612.f042.grib2', '2020/20200106/gfs.0p25.2020010612.f045.grib2', '2020/20200106/gfs.0p25.2020010612.f048.grib2', '2020/20200106/gfs.0p25.2020010612.f051.grib2', '2020/20200106/gfs.0p25.2020010612.f054.grib2', '2020/20200106/gfs.0p25.2020010612.f057.grib2', '2020/20200106/gfs.0p25.2020010612.f060.grib2', '2020/20200106/gfs.0p25.2020010612.f063.grib2', '2020/20200106/gfs.0p25.2020010612.f066.grib2', '2020/20200106/gfs.0p25.2020010612.f069.grib2', '2020/20200106/gfs.0p25.2020010612.f072.grib2', '2020/20200106/gfs.0p25.2020010612.f075.grib2', '2020/20200106/gfs.0p25.2020010612.f078.grib2', '2020/20200106/gfs.0p25.2020010612.f081.grib2', '2020/20200106/gfs.0p25.2020010612.f084.grib2', '2020/20200106/gfs.0p25.2020010612.f087.grib2', '2020/20200106/gfs.0p25.2020010612.f090.grib2', '2020/20200106/gfs.0p25.2020010612.f093.grib2', '2020/20200106/gfs.0p25.2020010612.f096.grib2', '2020/20200106/gfs.0p25.2020010612.f099.grib2', '2020/20200106/gfs.0p25.2020010612.f102.grib2', '2020/20200106/gfs.0p25.2020010612.f105.grib2', '2020/20200106/gfs.0p25.2020010612.f108.grib2', '2020/20200106/gfs.0p25.2020010612.f111.grib2', '2020/20200106/gfs.0p25.2020010612.f114.grib2', '2020/20200106/gfs.0p25.2020010612.f117.grib2', '2020/20200106/gfs.0p25.2020010612.f120.grib2', '2020/20200106/gfs.0p25.2020010612.f123.grib2', '2020/20200106/gfs.0p25.2020010612.f126.grib2', '2020/20200106/gfs.0p25.2020010612.f129.grib2', '2020/20200106/gfs.0p25.2020010612.f132.grib2', '2020/20200106/gfs.0p25.2020010612.f135.grib2', '2020/20200106/gfs.0p25.2020010612.f138.grib2', '2020/20200106/gfs.0p25.2020010612.f141.grib2', '2020/20200106/gfs.0p25.2020010612.f144.grib2', '2020/20200106/gfs.0p25.2020010612.f147.grib2', '2020/20200106/gfs.0p25.2020010612.f150.grib2', '2020/20200106/gfs.0p25.2020010612.f153.grib2', '2020/20200106/gfs.0p25.2020010612.f156.grib2', '2020/20200106/gfs.0p25.2020010612.f159.grib2', '2020/20200106/gfs.0p25.2020010612.f162.grib2', '2020/20200106/gfs.0p25.2020010612.f165.grib2', '2020/20200106/gfs.0p25.2020010612.f168.grib2', '2020/20200106/gfs.0p25.2020010612.f171.grib2', '2020/20200106/gfs.0p25.2020010612.f174.grib2', '2020/20200106/gfs.0p25.2020010612.f177.grib2', '2020/20200106/gfs.0p25.2020010612.f180.grib2', '2020/20200106/gfs.0p25.2020010612.f183.grib2', '2020/20200106/gfs.0p25.2020010612.f186.grib2', '2020/20200106/gfs.0p25.2020010612.f189.grib2', '2020/20200106/gfs.0p25.2020010612.f192.grib2', '2020/20200106/gfs.0p25.2020010612.f195.grib2', '2020/20200106/gfs.0p25.2020010612.f198.grib2', '2020/20200106/gfs.0p25.2020010612.f201.grib2', '2020/20200106/gfs.0p25.2020010612.f204.grib2', '2020/20200106/gfs.0p25.2020010612.f207.grib2', '2020/20200106/gfs.0p25.2020010612.f210.grib2', '2020/20200106/gfs.0p25.2020010612.f213.grib2', '2020/20200106/gfs.0p25.2020010612.f216.grib2', '2020/20200106/gfs.0p25.2020010612.f219.grib2', '2020/20200106/gfs.0p25.2020010612.f222.grib2', '2020/20200106/gfs.0p25.2020010612.f225.grib2', '2020/20200106/gfs.0p25.2020010612.f228.grib2', '2020/20200106/gfs.0p25.2020010612.f231.grib2', '2020/20200106/gfs.0p25.2020010612.f234.grib2', '2020/20200106/gfs.0p25.2020010612.f237.grib2', '2020/20200106/gfs.0p25.2020010612.f240.grib2', '2020/20200106/gfs.0p25.2020010612.f252.grib2', '2020/20200106/gfs.0p25.2020010612.f264.grib2', '2020/20200106/gfs.0p25.2020010612.f276.grib2', '2020/20200106/gfs.0p25.2020010612.f288.grib2', '2020/20200106/gfs.0p25.2020010612.f300.grib2', '2020/20200106/gfs.0p25.2020010612.f312.grib2', '2020/20200106/gfs.0p25.2020010612.f324.grib2', '2020/20200106/gfs.0p25.2020010612.f336.grib2', '2020/20200106/gfs.0p25.2020010612.f348.grib2', '2020/20200106/gfs.0p25.2020010612.f360.grib2', '2020/20200106/gfs.0p25.2020010612.f372.grib2', '2020/20200106/gfs.0p25.2020010612.f384.grib2', '2020/20200106/gfs.0p25.2020010618.f000.grib2', '2020/20200106/gfs.0p25.2020010618.f003.grib2', '2020/20200106/gfs.0p25.2020010618.f006.grib2', '2020/20200106/gfs.0p25.2020010618.f009.grib2', '2020/20200106/gfs.0p25.2020010618.f012.grib2', '2020/20200106/gfs.0p25.2020010618.f015.grib2', '2020/20200106/gfs.0p25.2020010618.f018.grib2', '2020/20200106/gfs.0p25.2020010618.f021.grib2', '2020/20200106/gfs.0p25.2020010618.f024.grib2', '2020/20200106/gfs.0p25.2020010618.f027.grib2', '2020/20200106/gfs.0p25.2020010618.f030.grib2', '2020/20200106/gfs.0p25.2020010618.f033.grib2', '2020/20200106/gfs.0p25.2020010618.f036.grib2', '2020/20200106/gfs.0p25.2020010618.f039.grib2', '2020/20200106/gfs.0p25.2020010618.f042.grib2', '2020/20200106/gfs.0p25.2020010618.f045.grib2', '2020/20200106/gfs.0p25.2020010618.f048.grib2', '2020/20200106/gfs.0p25.2020010618.f051.grib2', '2020/20200106/gfs.0p25.2020010618.f054.grib2', '2020/20200106/gfs.0p25.2020010618.f057.grib2', '2020/20200106/gfs.0p25.2020010618.f060.grib2', '2020/20200106/gfs.0p25.2020010618.f063.grib2', '2020/20200106/gfs.0p25.2020010618.f066.grib2', '2020/20200106/gfs.0p25.2020010618.f069.grib2', '2020/20200106/gfs.0p25.2020010618.f072.grib2', '2020/20200106/gfs.0p25.2020010618.f075.grib2', '2020/20200106/gfs.0p25.2020010618.f078.grib2', '2020/20200106/gfs.0p25.2020010618.f081.grib2', '2020/20200106/gfs.0p25.2020010618.f084.grib2', '2020/20200106/gfs.0p25.2020010618.f087.grib2', '2020/20200106/gfs.0p25.2020010618.f090.grib2', '2020/20200106/gfs.0p25.2020010618.f093.grib2', '2020/20200106/gfs.0p25.2020010618.f096.grib2', '2020/20200106/gfs.0p25.2020010618.f099.grib2', '2020/20200106/gfs.0p25.2020010618.f102.grib2', '2020/20200106/gfs.0p25.2020010618.f105.grib2', '2020/20200106/gfs.0p25.2020010618.f108.grib2', '2020/20200106/gfs.0p25.2020010618.f111.grib2', '2020/20200106/gfs.0p25.2020010618.f114.grib2', '2020/20200106/gfs.0p25.2020010618.f117.grib2', '2020/20200106/gfs.0p25.2020010618.f120.grib2', '2020/20200106/gfs.0p25.2020010618.f123.grib2', '2020/20200106/gfs.0p25.2020010618.f126.grib2', '2020/20200106/gfs.0p25.2020010618.f129.grib2', '2020/20200106/gfs.0p25.2020010618.f132.grib2', '2020/20200106/gfs.0p25.2020010618.f135.grib2', '2020/20200106/gfs.0p25.2020010618.f138.grib2', '2020/20200106/gfs.0p25.2020010618.f141.grib2', '2020/20200106/gfs.0p25.2020010618.f144.grib2', '2020/20200106/gfs.0p25.2020010618.f147.grib2', '2020/20200106/gfs.0p25.2020010618.f150.grib2', '2020/20200106/gfs.0p25.2020010618.f153.grib2', '2020/20200106/gfs.0p25.2020010618.f156.grib2', '2020/20200106/gfs.0p25.2020010618.f159.grib2', '2020/20200106/gfs.0p25.2020010618.f162.grib2', '2020/20200106/gfs.0p25.2020010618.f165.grib2', '2020/20200106/gfs.0p25.2020010618.f168.grib2', '2020/20200106/gfs.0p25.2020010618.f171.grib2', '2020/20200106/gfs.0p25.2020010618.f174.grib2', '2020/20200106/gfs.0p25.2020010618.f177.grib2', '2020/20200106/gfs.0p25.2020010618.f180.grib2', '2020/20200106/gfs.0p25.2020010618.f183.grib2', '2020/20200106/gfs.0p25.2020010618.f186.grib2', '2020/20200106/gfs.0p25.2020010618.f189.grib2', '2020/20200106/gfs.0p25.2020010618.f192.grib2', '2020/20200106/gfs.0p25.2020010618.f195.grib2', '2020/20200106/gfs.0p25.2020010618.f198.grib2', '2020/20200106/gfs.0p25.2020010618.f201.grib2', '2020/20200106/gfs.0p25.2020010618.f204.grib2', '2020/20200106/gfs.0p25.2020010618.f207.grib2', '2020/20200106/gfs.0p25.2020010618.f210.grib2', '2020/20200106/gfs.0p25.2020010618.f213.grib2', '2020/20200106/gfs.0p25.2020010618.f216.grib2', '2020/20200106/gfs.0p25.2020010618.f219.grib2', '2020/20200106/gfs.0p25.2020010618.f222.grib2', '2020/20200106/gfs.0p25.2020010618.f225.grib2', '2020/20200106/gfs.0p25.2020010618.f228.grib2', '2020/20200106/gfs.0p25.2020010618.f231.grib2', '2020/20200106/gfs.0p25.2020010618.f234.grib2', '2020/20200106/gfs.0p25.2020010618.f237.grib2', '2020/20200106/gfs.0p25.2020010618.f240.grib2', '2020/20200106/gfs.0p25.2020010618.f252.grib2', '2020/20200106/gfs.0p25.2020010618.f264.grib2', '2020/20200106/gfs.0p25.2020010618.f276.grib2', '2020/20200106/gfs.0p25.2020010618.f288.grib2', '2020/20200106/gfs.0p25.2020010618.f300.grib2', '2020/20200106/gfs.0p25.2020010618.f312.grib2', '2020/20200106/gfs.0p25.2020010618.f324.grib2', '2020/20200106/gfs.0p25.2020010618.f336.grib2', '2020/20200106/gfs.0p25.2020010618.f348.grib2', '2020/20200106/gfs.0p25.2020010618.f360.grib2']

Change this if you want your files saved somewhere other than the current directory 

In [ ]:
save_dir = './'
print(filelist)

 #### Now to download the files

In [ ]:
for file in filelist:
    filename = dspath + file
    outfile = save_dir + os.path.basename(filename)
    print('Downloading', file)
    req = requests.get(filename, cookies = ret.cookies, allow_redirects=True)
    open(outfile, 'wb').write(req.content)

### Once the data has been downloaded, you can begin to plotting.

In order to plot this data, you may need to install librariesIn order to plot this data, you may need to install libraries. The easiest way to do this is to use conda, however any method of getting the following libraries will work.

In [ ]:
import xarray # used for reading the data.
import matplotlib.pyplot as plt # used to plot the data.
import ipywidgets as widgets # For ease in selecting variables.
import cartopy.crs as ccrs # Used to georeference data.

In [ ]:
filelist_arr = [save_dir + os.path.basename(file) for file in filelist]
selected_file = widgets.Dropdown(options=filelist_arr, description='data file')
display(selected_file)

In [ ]:
# Now to load in the data to xarray
ds = xarray.open_dataset(selected_file.value)

In [ ]:
# Helper methods# Define function to get standard dimensions
def get_time(dataset):
    for _,cur_coord in dataset.coords.items:
        if cur_coord.attrs['standard_name'] == 'time':
            return cur_coord
def get_lat(dataset):
    for _,cur_coord in dataset.coords.items:
        if cur_coord.attrs['standard_name'] == 'longitude':
            return cur_coord
def get_lon(dataset):
    for _,cur_coord in dataset.coords.items:
        if cur_coord.attrs['standard_name'] == 'latitude':
            return cur_coord

def get_primary(dataset):
    primary_variables = {}
    coords = dataset.coords.keys()
    highest_dims = 0
    for cur_key,cur_var in dataset.variables.items():
        if cur_key not in coords:
            primary_variables[cur_key] = cur_var
    return primary_variables 

In [ ]:
var = widgets.Dropdown(
    options=get_primary(ds).keys(),
    description='Variable')
display(var)

In [ ]:
var = widgets.Dropdown(
proj = ccrs.Mercator()
plt.gcf().set_size_inches(20,10)
ax = plt.axes(projection=proj)
data_slice = ds[var.value].isel(time=10)
data_slice.plot.contourf(ax=ax, transform=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()